In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cat /proc/cpuinfo

In [ ]:
!/opt/bin/nvidia-smi


Fri Jul 14 12:27:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 安装 openai-whisper 的相关的依赖包

In [ ]:
%pip install openai-whisper
%pip install setuptools-rust

## Whisper本地转录


In [ ]:
import whisper
import time
import torch
# 释放显存空间。
torch.cuda.empty_cache()

model = whisper.load_model("large")
index = 11 # number of fi

def transcript_txt(clip, prompt, output):
    result = model.transcribe(clip, initial_prompt=prompt)
    with open(output, "w") as f:
        f.write(result['text'])
    print("Transcripted: ", clip)

original_prompt = "这是一段Onboard播客，里面会聊到ChatGPT以及PALM这个大语言模型。这个模型也叫做Pathways Language Model。\n\n"
prompt = original_prompt
for i in range(index):
    clip = f"./drive/MyDrive/colab_data/podcast/podcast_clip_{i}.mp3"
    output = f"./drive/MyDrive/colab_data/podcast/transcripts/local_podcast_clip_{i}.txt"
    start_time = time.time() # 获取开始时间
    transcript_txt(clip, prompt, output)
    # get last sentence of the transcript
    with open(output, "r") as f:
        transcript = f.read()
    end_time = time.time() # 获取结束时间
    print("运行时长(秒): ", "{:.3f}".format(end_time - start_time))
    sentences = transcript.split("。")
    prompt = original_prompt + sentences[-1]

100%|█████████████████████████████████████| 2.87G/2.87G [01:13<00:00, 42.3MiB/s]


Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_0.mp3
运行时长(秒):  243.446
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_1.mp3
运行时长(秒):  276.306
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_2.mp3
运行时长(秒):  252.416
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_3.mp3
运行时长(秒):  268.098
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_4.mp3
运行时长(秒):  256.330
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_5.mp3
运行时长(秒):  342.110
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_6.mp3
运行时长(秒):  334.612
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_7.mp3
运行时长(秒):  242.403
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_8.mp3
运行时长(秒):  243.447
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_9.mp3
运行时长(秒):  236.943
Transcripted:  ./drive/MyDrive/colab_data/podcast/podcast_clip_10.mp3
运行时长(秒):  9.217


## 下载B站音频，调用Whisper云端接口识别，然后用ChatGPT总结

In [ ]:
import torch
# 释放显存空间。
torch.cuda.empty_cache()

In [ ]:
%pip install pydub
%pip install openai
%pip install langchain
%pip install llama_index
!pip install spacy && python -m spacy download zh_core_web_sm

In [ ]:
!rm /content/data
!ln -s /content/drive/MyDrive/colab_data /content/data

rm: cannot remove '/content/data': No such file or directory


In [11]:
## 爬取B站音频，语音识别成文字，然后总结要点

import requests
import re
import json
import subprocess
import os
import argparse
import logging
from pydub import AudioSegment
import time
import shutil
import openai

from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import SpacyTextSplitter
from llama_index import GPTListIndex, LLMPredictor, ServiceContext, SimpleDirectoryReader
from llama_index import Prompt
from llama_index.node_parser import SimpleNodeParser

global video_title


# 缓存B站url音频
def download_audio(url):
    global video_title

    start_time = time.time() # 获取开始时间

    # B站视频网址
    # url = "https://www.bilibili.com/video/BV1Z4411179m/?spm_id_from=333.999.0.0"

    # 添加headers请求头，对Python解释器进行伪装
    # referer 和 User-Agent要改写成字典形式
    headers = {
        "referer":"https://www.bilibili.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }

    # 用 requests 的 get 方法访问网页
    response = requests.get(url=url, headers=headers)

    # 返回响应状态码：<Response [200]>
    # print_and_log("返回200，则网页请求成功：",response)

    # .text获取网页源代码
    # print_and_log(response.text)

    # 提取视频标题
    # 调用 re 的 findall 方法，去response.text中匹配我们要的标题
    # 正则表达式提取的数据返回的是一个列表，用[0]从列表中取值
    video_title = re.findall('<h1 title="(.*?)"', response.text)[0]
    # 如果标题里有[\/:*?<>|]特殊字符，直接删除
    video_title = re.sub(r"[\/:*?<>|]","",video_title)
    print_and_log("视频标题：",video_title)


    # type函数查看title的数据类型
    # print_and_log(type(title))

    # 提取 playinfo 里的数据
    # 调用 re的 findall 方法，去 response.text 中匹配我们要的数据
    # 正则表达式提取的数据返回的是一个列表，用[0]从列表中取值
    html_data =  re.findall('<script>window.__playinfo__=(.*?)</script>', response.text)[0]

    # html_data是字符串类型，将字符串转换成字典
    json_data=json.loads(html_data)

    # 让pycharm控制台以json格式化输出
    # 不影响程序，只改变pycharm或vscode编辑器的终端输出显示
    # indent=4 缩进4个空格
    json_dicts = json.dumps(json_data,indent=4)

    # print_and_log(json_dicts)

    # # 提取视频画面网址
    # video_url = json_data["data"]["dash"]["video"][0]["baseUrl"]
    # print("视频画面地址为：", video_url)
    # 提取音频网址
    audio_url = json_data["data"]["dash"]["audio"][0]["baseUrl"]
    print_and_log("音频地址：", audio_url)

    print_and_log("\n音频缓存中.......")

    # response.content获取响应体的二进制数据
    audio_content = requests.get(url=audio_url,headers=headers).content

    # # 创建mp3文件，写入二进制数据到硬盘
    audio_tmp_file = "./data/" + video_title + "_tmp.mp3"
    with open (audio_tmp_file, mode = "wb") as f :
        f.write(audio_content)
    # 将下载的音频数据保存到BytesIO对象中
    # audio_buffer = BytesIO()
    # audio_buffer.write(audio_content)

    print("音频缓存成功！")
    end_time = time.time() # 获取结束时间
    print("音频下载运行时长(秒): ", "{:.3f}".format(end_time - start_time))

    # 删除不需要的mp3
    #os.remove(audio_tmp_file)

    start_time = time.time() # 获取开始时间
    # # 转换mp3格式
    mp3_file = "./data/" + video_title + ".mp3"
    cmd = f"ffmpeg -y -i {audio_tmp_file} -vn -c:a libmp3lame -q:a 4 {mp3_file}"
    subprocess.run(cmd,shell=True)
    print_and_log("音频转换成功！")
    end_time = time.time() # 获取结束时间
    print_and_log("格式转换运行时长(秒): ", "{:.3f}".format(end_time - start_time))
    return mp3_file


# 长音频切分成短音频
def audio_cut(mp3_file):
    global video_title

    print_and_log("\n音频切分中.......")
    start_time = time.time() # 获取开始时间

    long_file = AudioSegment.from_mp3(mp3_file)

    # PyDub handles time in milliseconds
    ten_minutes = 15 * 60 * 1000 # 15分钟

    total_length = len(long_file)

    start = 0
    index = 0
    while start < total_length:
        end = start + ten_minutes
        if end < total_length:
            chunk = long_file[start:end]
        else:
            chunk = long_file[start:]
        with open(f"./data/{video_title}_{index}.mp3", "wb") as f:
            chunk.export(f, format="mp3")
        start = end
        index += 1

    end_time = time.time() # 获取结束时间
    print_and_log("音频切分运行时长(秒): ", "{:.3f}".format(end_time - start_time))

    return index


# 语音识别
def speech_txt(index):
    global video_title

    print_and_log("\n语音识别中.......")
    start_time = time.time() # 获取开始时间
    # 用whisper进行语音识别，注意Prompt增加中英文混排格式和标点符号

    openai.api_key = os.getenv("OPENAI_API_KEY")

    original_prompt = video_title + "。"
    prompt = original_prompt

    srt_dir = f"./data/srt"
    if os.path.isdir(srt_dir):
        shutil.rmtree(srt_dir)
    os.makedirs(srt_dir)

    txt_dir = f"./data/txt"
    if os.path.isdir(txt_dir):
        shutil.rmtree(txt_dir)
    os.makedirs(txt_dir)

    for i in range(index):
        print_and_log("index: ", i)
        clip = f"./data/{video_title}_{i}.mp3"
        fp_audio= open(clip, "rb")
        print_and_log("audio file: ", clip)
        start_time1 = time.time() # 获取开始时间
        transcript = openai.Audio.transcribe("whisper-1", fp_audio, response_format="srt",
                                        prompt=prompt)
        end_time = time.time() # 获取结束时间
        print_and_log("语音识别运行时长(秒): ", "{:.3f}".format(end_time - start_time1))

        # 保存字幕文本到文件，包含时间信息
        with open(f"{srt_dir}/{video_title}_{i}.srt", "w", encoding='utf-8') as f:
            f.write(transcript)
        # get last sentence of the transcript
        sentences = transcript.split("。")
        prompt = original_prompt + sentences[-1]

        # 保存字幕文本到文件，不包含时间信息
        with open(f"{txt_dir}/{video_title}_{i}.txt", "w", encoding='utf-8') as f:
            lines = transcript.strip().split("\n")
            content = "\n".join(lines[2::4])  # 跳过时间行，仅保存文本行
            f.write(content)
        

    print_and_log("语音识别完成！")
    end_time = time.time() # 获取结束时间
    print_and_log("语音识别运行总时长(秒): ", "{:.3f}".format(end_time - start_time))
    return txt_dir


# 用ChatGPT总结
def gtp_summarize(txt_dir):
    global video_title

    print_and_log("\n文本总结中.......")
    start_time = time.time() # 获取开始时间
    # define LLM
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=1024))

    text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 4096)
    parser = SimpleNodeParser(text_splitter=text_splitter)
    documents = SimpleDirectoryReader(txt_dir).load_data()
    nodes = parser.get_nodes_from_documents(documents)

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

    list_index = GPTListIndex(nodes=nodes, service_context=service_context)
    # tree_summarize （总结摘要-最优）
    # simple_summarize （最省token）
    # refine （基于关键词询问-最优）
    # compact（较省token）
    query_engine = list_index.as_query_engine(response_mode="tree_summarize")

    # response = query_engine.query("请你总结一下内容，要点限定在5条以内，并以Markdown格式显示")
    prompt = "该视频的标题是：" + video_title + "，请你总结一下内容要点，尽量简洁！"
    response = query_engine.query(prompt)

    # 总结
    print_and_log("视频标题：", video_title)
    print_and_log("视频要点：", response)
    end_time = time.time() # 获取结束时间
    print_and_log("文本总结运行时长(秒): ", "{:.3f}".format(end_time - start_time))
    return response


# 配置日志
log_file = "./data/logout.log"
print(log_file)
logging.basicConfig(filename=log_file, level=logging.DEBUG, format='%(asctime)s %(message)s', encoding='utf-8')
# 打印输出并保存到日志文件
def print_and_log(*args):
    message = ""
    if len(args) == 1 and isinstance(args[0], str):
        # 单个参数且为字符串，进行格式化输出
        message = args[0]
    else:
        # 多个参数或参数类型不为字符串，进行拼接输出
        message = ' '.join(str(arg) for arg in args)

    print(message)
    logging.debug(message)

# 主函数
if __name__ == "__main__":
    # parser = argparse.ArgumentParser()
    # parser.add_argument('-u', '--url', required=False, help='B站视频url')
    # args = parser.parse_args()
    # url = args.url

    # url = "https://www.bilibili.com/video/BV1Z4411179m/?spm_id_from=333.999.0.0"
    url = "https://www.bilibili.com/video/BV1xE411B7V4/?spm_id_from=333.337.search-card.all.click"

    start_time = time.time() # 获取开始时间

    # 缓存音频
    #mp3_file = download_audio(url)

    # 切分音频
    #index = audio_cut(mp3_file)
    index = 2

    # 语音识别
    txt_dir = speech_txt(index)

    # 文本总结
    summarize_txt = gtp_summarize(txt_dir)

    end_time = time.time() # 获取结束时间

    # 计算总时长
    mp3_second = len(AudioSegment.from_file(mp3_file)) / 1000  # 获取总毫秒数并转换为秒
    print_and_log("音频总时长(秒): ", mp3_second)
    print_and_log("运行总时长(秒): ", "{:.3f}".format(end_time - start_time))



./data/logout.log
视频标题： 如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】
音频地址： https://upos-sz-mirroraliov.bilivideo.com/upgcxcode/87/56/127675687/127675687_nb2-1-30280.m4s?e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M=&uipk=5&nbs=1&deadline=1689393203&gen=playurlv2&os=aliovbv&oi=575673980&trid=046d2ae760004047bae3b72198749263u&mid=0&platform=pc&upsig=fba81eeca7135102c9f809fae4478046&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,mid,platform&bvc=vod&nettype=0&orderid=0,1&buvid=&build=0&agrr=1&bw=24536&logo=80000000

音频缓存中.......
音频缓存成功！
音频下载运行时长(秒):  38.109
音频转换成功！
格式转换运行时长(秒):  29.448

音频切分中.......
音频切分运行时长(秒):  37.043

语音识别中.......
index:  0
audio file:  ./data/如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】_0.mp3
语音识别运行时长(秒):  65.261


TypeError: ignored

## log保存测试

In [ ]:
import os
import logging

# 配置日志
log_file = '/content/data/logout.txt'
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s', handlers=[
    logging.FileHandler(log_file),
    logging.StreamHandler()
])
# logging.basicConfig(filename=log_file, level=logging.DEBUG, format='%(asctime)s %(message)s', encoding='utf-8')

# 打印输出并保存到日志文件
def print_and_log(*args):
    message = ""
    if len(args) == 1 and isinstance(args[0], str):
        # 单个参数且为字符串，进行格式化输出
        message = args[0]
    else:
        # 多个参数或参数类型不为字符串，进行拼接输出
        message = ' '.join(str(arg) for arg in args)

    print(message)
    logging.debug(message)
print_and_log('hello world!\n')

## Whisper本地转录测试

In [ ]:
import torch
# 释放显存空间。
torch.cuda.empty_cache()

In [ ]:
import whisper
import time
import torch
# 释放显存空间。
torch.cuda.empty_cache()

model = whisper.load_model("small")
index = 2 # number of file

def transcript_txt(clip, prompt, output):
    result = model.transcribe(clip, initial_prompt=prompt)
    with open(output, "w") as f:
        f.write(result['text'])
    print("Transcripted: ", clip)

original_prompt = "如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】。\n\n"
prompt = original_prompt
for i in range(index):
    clip = f"./data/如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】_{i}.mp3"
    output = f"./data/如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】_{i}.txt"
    start_time = time.time() # 获取开始时间
    transcript_txt(clip, prompt, output)
    # get last sentence of the transcript
    with open(output, "r") as f:
        transcript = f.read()
    end_time = time.time() # 获取结束时间
    print("运行时长(秒): ", "{:.3f}".format(end_time - start_time))
    sentences = transcript.split("。")
    prompt = original_prompt + sentences[-1]

100%|███████████████████████████████████████| 461M/461M [01:09<00:00, 6.94MiB/s]


Transcripted:  ./data/如何才能摆脱贫穷？穷人和富人有什么差别？【诺贝尔经济学奖解读】_0.mp3
运行时长(秒):  122.239
